In [159]:
import pandas as pd

In [160]:
ua = pd.read_csv('ua.base', delimiter='\t', iterator=False, names=['user','item','ratings','timestamp'])

In [161]:
item = pd.read_csv('u.item', delimiter='|', encoding='ISO-8859-1', names=['item','title','release_date', \
                                                                         'video_release_date', 'IMDB_URL','unknown','Action','Adventure',\
                                                                         'Animation','Childrens','Comedy','Crime','Documentary','Drama', \
                                                                         'Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi', \
                                                                         'Thriller','War','Western'])

In [162]:
item.head()

,item,title,release_date,video_release_date,IMDB_URL,unknown,Action,Adventure,Animation,Childrens,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [165]:
# 处理title
import gensim, logging,re 
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
item['title'] = item.title.apply(lambda x: re.sub('\\(.*?\\)|\\{.*?}|\\[.*?]|(.*?)','',x))
titles = item['title']

In [170]:
from nltk.corpus import stopwords
wnl = WordNetLemmatizer() 
sentences = [title.strip().split(' ') for title in titles]
# 提取词干
title_set = set([ word for words in sentences for word in words])
title2map = {}
from nltk.stem import WordNetLemmatizer 
wnl = WordNetLemmatizer() 
title2map = {v:wnl.lemmatize(v.lower()) for k,v in enumerate(title_set)}
# 去除停顿词
from nltk.corpus import stopwords 
stop = set(stopwords.words('english')) 
filtered_sentences = []
for sentence in sentences:
    ans = []
    for word in sentence:
        if not title2map[word] in stop:
            ans.append(title2map[word])
    filtered_sentences.append(ans)

In [175]:
model = gensim.models.Word2Vec(filtered_sentences, min_count=1, size=10)

2019-05-31 15:30:32,825 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2019-05-31 15:30:32,826 : INFO : collecting all words and their counts
2019-05-31 15:30:32,826 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-05-31 15:30:32,828 : INFO : collected 2267 word types from a corpus of 3422 raw words and 1682 sentences
2019-05-31 15:30:32,830 : INFO : Loading a fresh vocabulary
2019-05-31 15:30:32,834 : INFO : effective_min_count=1 retains 2267 unique words (100% of original 2267, drops 0)
2019-05-31 15:30:32,836 : INFO : effective_min_count=1 leaves 3422 word corpus (100% of original 3422, drops 0)
2019-05-31 15:30:32,845 : INFO : deleting the raw counts dictionary of 2267 items
2019-05-31 15:30:32,845 : INFO : sample=0.001 downsamples 20 most-common words
2019-05-31 15:30:32,846 : INFO : downsampling leaves estimated 3350 word corpus (97.9% of prior 3422)
2019-05-31 15:30:32,851 : INFO : estimated required memory for 2

In [179]:
for sentence in sentences:
    ans = []
    for word in sentence:
        if not title2map[word] in stop:
            ans.append(title2map[word])
    filtered_sentences.append(ans)

/Users/dennis/.conda/envs/Reclib/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-0.00918109, -0.02676832,  0.02179421, -0.01802769, -0.03800369,
        0.00241234,  0.01812018,  0.00902718,  0.01317871, -0.03705009],
      dtype=float32)

In [ ]:
item = item.drop(['video_release_date'], axis=1)

In [ ]:
item.head()

In [ ]:
item = item.fillna(-1)

In [ ]:
ua = pd.merge(ua, item,how='left', on='item')

In [ ]:
ua = ua.drop(['IMDB_URL'], axis=1)

In [ ]:
user = pd.read_csv('u.user', delimiter='|', names=['user','age','gender','occupation','zip_code'])

In [ ]:
user.head()

In [ ]:
from sklearn.feature_extraction import DictVectorizer